In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd 'drive/My Drive/PCPPN'
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/PCPPN


device(type='cuda')

## Imports

In [ ]:
pip install helpers

In [ ]:
target_test_accu = 0.01

import os
import shutil

import torch
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import argparse
import re

from helpers import makedir
import model
import push
import prune
import train_and_test as tnt
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function

from settings import base_architecture, img_size, prototype_shape, num_classes, \
                     prototype_activation_function, add_on_layers_type, experiment_run
print(prototype_shape)
print(num_classes)

(16, 128, 1, 1)
8


## Load Data

In [ ]:
#epoches_accuracy_saved = [i for i in range(1000) if i % 5 == 0] # show the epches to the accuracy logs be saved
# book keeping namings and code

from settings import n_fold

base_architecture_type = re.match('^[a-z]*', base_architecture).group(0)

model_dir = './saved_models'+'_'+'F'+str(n_fold)+'_'+str(num_classes)+'/' + base_architecture + '/' + experiment_run + '/'
makedir(model_dir)

# shutil.copy(src=os.path.join(os.getcwd(), __file__), dst=model_dir)  # I think this line should not be implemented (__file__ doesn't exist!)
shutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir) # Directory for saving prototypes
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

# load the data
from settings import train_dir, test_dir, train_push_dir, \
                     train_batch_size, test_batch_size, train_push_batch_size

# test_dir = train_push_dir
normalize = transforms.Normalize(mean=mean,
                                 std=std)

# all datasets
# train set

train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_loader = torch.utils.data.DataLoader(
    train_dataset , batch_size=train_batch_size, shuffle=True,
    num_workers=4, pin_memory=False)

# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

# we should look into distributed sampler more carefully at torch.utils.data.distributed.DistributedSampler(train_dataset)
log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
log('batch size: {0}'.format(train_batch_size))

training set size: 9600
push set size: 995
test set size: 605
batch size: 80


## Train Model

In [ ]:
1## Ok

epoches_accuracy_saved = [i for i in range(1000) if i % 10 == 0] # show the epches to the accuracy logs be saved
# book keeping namings and code
#from settings import base_architecture, img_size, prototype_shape, num_classes, n_fold, \
#                     prototype_activation_function, add_on_layers_type, experiment_run

base_architecture_type = re.match('^[a-z]*', base_architecture).group(0)

model_dir = './saved_models'+'_'+'F'+str(n_fold)+'_'+str(num_classes)+'/' + base_architecture + '/' + experiment_run + '/'
makedir(model_dir)

# shutil.copy(src=os.path.join(os.getcwd(), __file__), dst=model_dir)  # I think this line should not be implemented (__file__ doesn't exist!)
shutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir) # Directory for saving prototypes
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

# load the data
from settings import train_dir, test_dir, train_push_dir, \
                     train_batch_size, test_batch_size, train_push_batch_size

# test_dir = train_push_dir
normalize = transforms.Normalize(mean=mean,
                                 std=std)

# all datasets
# train set

train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_loader = torch.utils.data.DataLoader(
    train_dataset , batch_size=train_batch_size, shuffle=True,
    num_workers=4, pin_memory=False)

# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)





# we should look into distributed sampler more carefully at torch.utils.data.distributed.DistributedSampler(train_dataset)
log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
log('batch size: {0}'.format(train_batch_size))

# construct the model
ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)
#if prototype_activation_function == 'linear':
#    ppnet.set_last_layer_incorrect_connection(incorrect_strength=0)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)  # ??
class_specific = True

# define optimizer
from settings import joint_optimizer_lrs, joint_lr_step_size
joint_optimizer_specs = \
[{'params': ppnet.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.1)

from settings import warm_optimizer_lrs
warm_optimizer_specs = \
[{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)

from settings import last_layer_optimizer_lr
last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

# weighting of different training losses
from settings import coefs

# number of training epochs, number of warm epochs, push start epoch, push epochs
from settings import num_train_epochs, num_warm_epochs, push_start, push_epochs

test_epochs = [i for i in range(num_train_epochs) if i % 5 == 0]
# train the model
log('start training')
import copy

train_acc_log = []
test_acc_log = []
train_loss_log = []
test_loss_log = []

for epoch in range(num_train_epochs):
    log('epoch: \t{0}'.format(epoch))

    if epoch < num_warm_epochs:
        tnt.warm_only(model=ppnet_multi, log=log)
        acc_train, loss_train = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=warm_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)
    else:
        tnt.joint(model=ppnet_multi, log=log)
        joint_lr_scheduler.step()
        acc_train, loss_train = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=joint_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)
    accu, loss = tnt.test(model=ppnet_multi, dataloader=test_loader,
                    class_specific=class_specific, log=log)
    if epoch >= 5 and epoch in test_epochs:
      save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'nopush', accu=accu,
                                  target_accu = target_test_accu, log=log)

    train_acc_log.append(acc_train)
    test_acc_log.append(accu)
    train_loss_log.append(loss_train)
    test_loss_log.append(loss)

    if epoch >= push_start and epoch in push_epochs:
        push.push_prototypes(
            train_push_loader, # pytorch dataloader (must be unnormalized in [0,1])
            prototype_network_parallel=ppnet_multi, # pytorch network with prototype_vectors
            class_specific=class_specific,
            preprocess_input_function=preprocess_input_function, # normalize if needed
            prototype_layer_stride=1,
            root_dir_for_saving_prototypes=img_dir, # if not None, prototypes will be saved here
            epoch_number=epoch, # if not provided, prototypes saved previously will be overwritten
            prototype_img_filename_prefix=prototype_img_filename_prefix,
            prototype_self_act_filename_prefix=prototype_self_act_filename_prefix,
            proto_bound_boxes_filename_prefix=proto_bound_boxes_filename_prefix,
            save_prototype_class_identity=True,
            log=log)
        accu, loss = tnt.test(model=ppnet_multi, dataloader=test_loader,
                        class_specific=class_specific, log=log)
        save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'push', accu=accu,
                                    target_accu = target_test_accu, log=log)

        if prototype_activation_function != 'linear':
            tnt.last_only(model=ppnet_multi, log=log)
            for i in range(20):
                log('iteration: \t{0}'.format(i))
                acc_train, loss_train = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=last_layer_optimizer,
                              class_specific=class_specific, coefs=coefs, log=log)
                accu, loss = tnt.test(model=ppnet_multi, dataloader=test_loader,
                                class_specific=class_specific, log=log)
                save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + '_' + str(i) + 'push', accu=accu,
                                            target_accu = target_test_accu, log=log)

                train_acc_log.append(acc_train)
                test_acc_log.append(accu)
                train_loss_log.append(loss_train)
                test_loss_log.append(loss)
    if epoch >= 10 and epoch in epoches_accuracy_saved:
       #torch.save(train_acc_log,test_acc_log,train_loss_log,test_loss_log, 'Train_test_loss_log_f_'+str(n_fold)+'_c_'+str(num_classes)+'.pt')
       torch.save(train_acc_log, 'Train_log_f_'+str(n_fold)+'_c_'+str(num_classes)+'.pt')
       torch.save(test_acc_log, 'Test_log_f_'+str(n_fold)+'_c_'+str(num_classes)+'.pt')
       torch.save(train_loss_log, 'Train_log_loss_f_'+str(n_fold)+'_c_'+str(num_classes)+'.pt')
       torch.save(test_loss_log, 'Test_log_loss_f_'+str(n_fold)+'_c_'+str(num_classes)+'.pt')

logclose()

training set size: 9600
push set size: 995
test set size: 605
batch size: 80


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to ./pretrained_models/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 218MB/s]


start training
epoch: 	0
	warm
	train
	time: 	1121.6088812351227
	cross ent: 	2.7016143759091693
	cluster: 	3.315107613305251
	separation:	2.5213141376773516
	avg separation:	3.5812159637610117
	accu: 		12.28125%
	loss: 	5.163995480537414
	l1: 		136.0
	p dist pair: 	4.16764497756958
	test
	time: 	100.7240731716156
	cross ent: 	2.815673657826015
	cluster: 	0.872805757181985
	separation:	0.3090490017618452
	avg separation:	1.0081316658428736
	accu: 		10.743801652892563%
	loss: 	3.501194408961705
	l1: 		136.0
	p dist pair: 	4.16764497756958
epoch: 	1
	warm
	train
	time: 	105.04874348640442
	cross ent: 	2.4338632484277087
	cluster: 	0.3780346568673849
	separation:	0.19001649164905152
	avg separation:	0.6726459364096323
	accu: 		21.5%
	loss: 	2.7330897291501364
	l1: 		136.0
	p dist pair: 	2.163562774658203
	test
	time: 	9.51222276687622
	cross ent: 	2.789089100701468
	cluster: 	0.3541971913405827
	separation:	0.1254606917500496
	avg separation:	0.4617058038711548
	accu: 		10.082644628099173

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


	time: 	297.55283093452454
	cross ent: 	2.584975556532542
	cluster: 	0.18570576440542935
	separation:	0.09569033306712905
	avg separation:	0.27470331837733586
	accu: 		12.1875%
	loss: 	2.7378850251436235
	l1: 		136.0
	p dist pair: 	0.6077750325202942
	test
	time: 	9.78029990196228
	cross ent: 	2.6290575776781355
	cluster: 	0.11358606496027537
	separation:	0.056909658546958654
	avg separation:	0.2094301198210035
	accu: 		9.421487603305785%
	loss: 	2.7273737362452914
	l1: 		136.0
	p dist pair: 	0.6077750325202942
	above 1.00%
epoch: 	6
	joint
	train
	time: 	297.07749676704407
	cross ent: 	2.39928457736969
	cluster: 	0.08098789968838295
	separation:	0.046417815672854586
	avg separation:	0.24526401087641717
	accu: 		12.572916666666668%
	loss: 	2.4723615487416586
	l1: 		136.0
	p dist pair: 	0.632267415523529
	test
	time: 	9.718201160430908
	cross ent: 	2.4956657205309187
	cluster: 	0.07889885881117412
	separation:	0.03965476926948343
	avg separation:	0.2910814370427813
	accu: 		11.239669421

# Compute the Confusion Matrix

In [ ]:
# Set the trained model name here
model_name = '30nopush0.2066.pth'   #"30nopush0.1124.pth"

model_dir = './saved_models'+'_'+'F'+str(n_fold)+'_'+str(num_classes)+'/' + base_architecture + '/' + experiment_run + '/' # _Fold1_8
PATH = model_dir + model_name
test_batch_size = 1  # Note! When we iteratively load and process batches of data, large batch sizes may cause the CUDA to be out of memory
train_batch_size = 1
train_push_batch_size = 1
# book keeping namings and code
#from settings import base_architecture, img_size, prototype_shape, num_classes, \
#                     prototype_activation_function, add_on_layers_type, experiment_run

base_architecture_type = re.match('^[a-z]*', base_architecture).group(0)
%clear model, ppnet
# construct the model
ppnet = model.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)
#if prototype_activation_function == 'linear':
#    ppnet.set_last_layer_incorrect_connection(incorrect_strength=0)
ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)  # ??
class_specific = True

model = ppnet #_multi
model = torch.load(PATH)
#model = model.cuda()

shutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir) # Directory for saving prototypes
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

# load the data
from settings import train_dir, test_dir, train_push_dir
normalize = transforms.Normalize(mean=mean,
                                 std=std)
# all datasets
# train set
train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)
# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
    ]))
train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)
# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)
# we should look into distributed sampler more carefully at torch.utils.data.distributed.DistributedSampler(train_dataset)
log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
log('batch size: {0}'.format(train_batch_size))



training set size: 9600
push set size: 995
test set size: 605
batch size: 1


In [ ]:
# Test with 2*n_cluster pseudo-classes
#from settings import num_classes
dataloader = test_loader

#model = model.cuda()
All_preds = []
All_labels = []
model.eval()
#s_b = [0, 1, 2, 3, 4]
#s_m = [5, 6, 7, 8, 9]
#s_b = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 23, 34, 35, 36, 37, 38, 39]
#s_m = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
c_b = 0
c_m = 0
count = torch.zeros(40)
for i, (image, label) in enumerate(dataloader):
    input = image.cuda()
    target = label.cuda()
    count[label] += 1

    output, min_distances = model(input)
    _, predicted_0 = torch.max(output.data, 1)
    predicted = predicted_0.cpu().numpy().astype(int)

    #if target in s_b:  # # for original two classes in test
    if target < num_classes/2:
       c_b += 1
    else:
       c_m += 1

    All_preds.append(predicted)
    All_labels.append(target.cpu().numpy().astype(int))

    #c += 1
    #if c == 5:
    #   break

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(All_labels ,All_preds ))
print(classification_report(All_labels ,All_preds ))
#torch.save(All_preds, 'All_preds_10.pt')
#torch.save(All_labels, 'All_labels.pt')
print('All_labels:', All_labels)
print('All_preds:', All_preds)
print('Counts(b & m):', [c_b, c_m])

[[10  7  9 28  2  1  5  3]
 [ 8  5 10 18  4  1 10  1]
 [ 8  8  9 20  5  1  5  0]
 [14  5 10 22  4  2  2  2]
 [ 1  3  0  1 22 21 23 18]
 [ 0  2  0  1 26 20 26 27]
 [ 0  3  0  2 18 17 21 26]
 [ 1  5  0  0 21 21 25 15]]
              precision    recall  f1-score   support

           0       0.24      0.15      0.19        65
           1       0.13      0.09      0.11        57
           2       0.24      0.16      0.19        56
           3       0.24      0.36      0.29        61
           4       0.22      0.25      0.23        89
           5       0.24      0.20      0.22       102
           6       0.18      0.24      0.21        87
           7       0.16      0.17      0.17        88

    accuracy                           0.20       605
   macro avg       0.21      0.20      0.20       605
weighted avg       0.21      0.20      0.20       605

All_labels: [array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]), array([0]),

In [ ]:
# Test_acc with binarized pseudo_classes (remapped to B and M)

dataloader = test_loader  # train_loader  #

#model = model.cuda()
All_preds = []
All_labels = []
All_preds_2 = []
All_labels_2 = []
model.eval()
#s_b = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 23, 34, 35, 36, 37, 38, 39]
#s_m = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
c_b = 0
c_m = 0
count = torch.zeros(40)
for i, (image, label) in enumerate(dataloader):
    input = image.cuda()
    target_0 = label.cuda()
    count[label] += 1

    output, min_distances = model(input)
    _, predicted_0 = torch.max(output.data, 1)
    predicted_0 = predicted_0.cpu().numpy().astype(int)
    #predicted = predicted_0  # for 40 new classes in test
    #if predicted_0 in s_b:  # # for original two classes in test
    if predicted_0 < num_classes/2:
       predicted = 0
    else:
       predicted = 1

    #if target_0 in s_b:  # # for original two classes in test
    if target_0 < num_classes/2:
       target = 0
       c_b += 1
    else:
       target = 1
       c_m += 1

    All_preds.append(predicted)
    All_labels.append(target)

    All_preds_2.append(predicted_0)
    All_labels_2.append(target_0)

    #c += 1
    #if c == 5:
    #   break

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(All_labels ,All_preds ))
print(classification_report(All_labels ,All_preds ))
#torch.save(All_preds, 'All_preds_10.pt')
#torch.save(All_labels, 'All_labels.pt')
print('All_labels:', All_labels)
print('All_preds:', All_preds)
#import numpy as np
#print('difference:', np.array(All_labels) - np.array(All_preds))
print('Counts(b & m):', [c_b, c_m])

[[191  48]
 [ 19 347]]
              precision    recall  f1-score   support

           0       0.91      0.80      0.85       239
           1       0.88      0.95      0.91       366

    accuracy                           0.89       605
   macro avg       0.89      0.87      0.88       605
weighted avg       0.89      0.89      0.89       605

All_labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Train_acc with binarized pseudo_classes (remapped to B and M)

dataloader = train_loader  # train_loader  #

#model = model.cuda()
All_preds = []
All_labels = []
All_preds_2 = []
All_labels_2 = []
model.eval()
#s_b = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 23, 34, 35, 36, 37, 38, 39]
#s_m = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
c_b = 0
c_m = 0
count = torch.zeros(40)
for i, (image, label) in enumerate(dataloader):
    input = image.cuda()
    target_0 = label.cuda()
    count[label] += 1

    output, min_distances = model(input)
    _, predicted_0 = torch.max(output.data, 1)
    predicted_0 = predicted_0.cpu().numpy().astype(int)
    #predicted = predicted_0  # for 40 new classes in test
    #if predicted_0 in s_b:  # # for original two classes in test
    if predicted_0 < num_classes/2:
       predicted = 0
    else:
       predicted = 1

    #if target_0 in s_b:  # # for original two classes in test
    if target_0 < num_classes/2:
       target = 0
       c_b += 1
    else:
       target = 1
       c_m += 1

    All_preds.append(predicted)
    All_labels.append(target)

    All_preds_2.append(predicted_0)
    All_labels_2.append(target_0)

    #c += 1
    #if c == 5:
    #   break

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(All_labels ,All_preds ))
print(classification_report(All_labels ,All_preds ))
#torch.save(All_preds, 'All_preds_10.pt')
#torch.save(All_labels, 'All_labels.pt')
print('All_labels:', All_labels)
print('All_preds:', All_preds)
#import numpy as np
#print('difference:', np.array(All_labels) - np.array(All_preds))
print('Counts(b & m):', [c_b, c_m])

[[4792    8]
 [   8 4792]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4800
           1       1.00      1.00      1.00      4800

    accuracy                           1.00      9600
   macro avg       1.00      1.00      1.00      9600
weighted avg       1.00      1.00      1.00      9600

All_labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 